In [34]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import os
from outlier_removal.utils import estimating_missing_measuremnts, cumulative_calculation, outlier_removal_other_parameters, outlier_removal_species, remove_gluconeogenesis, delta_cumulative_values_calculation, outlier_detector_1st_order

In [35]:
# read data
df = pd.read_excel("input_files/outliers_removal/genentech_12000L_raw_after_mannual_removal.xlsx")
# get column names
df.columns

Index(['Date (MM/DD/YY H:MM:SS AM/PM)', 'Cell Line', 'ID', 'Name',
       'Initial Volume (mL)', 'Sample Volume (mL)',
       'Volume Before Sampling (mL)', 'Volume After Sampling (mL)',
       'Feed Media Added (mL)', 'Base Added (mL)', 'Osmolarity',
       'Viable Cell Concentration (10^6 cells/mL)',
       'Dead Cell Concentration (10^6 cells/mL)',
       'Total Cell Concentration (10^6 cells/mL)', 'IgG (mg/L)', 'NH4+', 'Na+',
       'Titer', 'IntvPCV', 'PCV', 'PCV.1', 'pCO2', 'pHoff', 'pO2',
       'air sparge', 'air sparge sp', 'air sparge total', 'backpressure',
       'base totalized', 'co2 sparge', 'co2 sparge total',
       'do2 controller output', 'do2 primary', 'do2 secondary',
       'flowrate overlay', 'oxygen sparge', 'oxygen sparge total',
       'ph controller output', 'ph primary', 'pressure exhaust valve',
       'sparge total', 'temperature', 'temperature jacket', 'weight load cell',
       'Titer_range', 'Titer_group', 'Titer_category', 'Glucose (mM)',
       'Lacta

In [36]:
df.columns

Index(['Date (MM/DD/YY H:MM:SS AM/PM)', 'Cell Line', 'ID', 'Name',
       'Initial Volume (mL)', 'Sample Volume (mL)',
       'Volume Before Sampling (mL)', 'Volume After Sampling (mL)',
       'Feed Media Added (mL)', 'Base Added (mL)', 'Osmolarity',
       'Viable Cell Concentration (10^6 cells/mL)',
       'Dead Cell Concentration (10^6 cells/mL)',
       'Total Cell Concentration (10^6 cells/mL)', 'IgG (mg/L)', 'NH4+', 'Na+',
       'Titer', 'IntvPCV', 'PCV', 'PCV.1', 'pCO2', 'pHoff', 'pO2',
       'air sparge', 'air sparge sp', 'air sparge total', 'backpressure',
       'base totalized', 'co2 sparge', 'co2 sparge total',
       'do2 controller output', 'do2 primary', 'do2 secondary',
       'flowrate overlay', 'oxygen sparge', 'oxygen sparge total',
       'ph controller output', 'ph primary', 'pressure exhaust valve',
       'sparge total', 'temperature', 'temperature jacket', 'weight load cell',
       'Titer_range', 'Titer_group', 'Titer_category', 'Glucose (mM)',
       'Lacta

In [37]:
# get the list of unique datetimes
datetime_lst = df['Date (MM/DD/YY H:MM:SS AM/PM)'].unique().tolist()

# get the list of run IDs
runID_lst = df['ID'].unique().tolist()
# runID_drop_lst = runID_lst[[149,218]]

# run 149, 218 have very high initial lactate concentration

# add new colum for feeding indicator
df['feed bool'] = (df['Glucose (mM)'] != df['Glucose after feeding (mM)']).tolist()

feed_idx_lst = df[(df['Glucose (mM)'] != df['Glucose after feeding (mM)'])].index.tolist()


# chose the columns to remove outliers
cols_lst = [
        # 'Date (MM/DD/YY H:MM:SS AM/PM)', 'Cell Line', 'ID', 'Name',
    #    'Initial Volume (mL)', 'Sample Volume (mL)',
    #    'Volume Before Sampling (mL)', 'Volume After Sampling (mL)',
    #    'Feed Media Added (mL)', 'Feed Status', '# Feed', 'Base Added (mL)',
       'Osmolarity', 'Viable Cell Concentration (10^6 cells/mL)',
       'Dead Cell Concentration (10^6 cells/mL)',
       'Total Cell Concentration (10^6 cells/mL)'#, 'IgG (mg/L)', 
       'NH4+', 'Na+',
    #    'Titer', 'IntvPCV', 'PCV', 'pCO2', 'pHoff', 'pO2', 'air sparge',
    #    'air sparge sp', 'air sparge total', 'backpressure', 'co2 sparge',
    #    'co2 sparge total', 'do2 controller output', 'do2 primary',
    #    'do2 secondary', 'flowrate overlay', 'oxygen sparge',
    #    'oxygen sparge total', 'ph controller output', 'ph primary',
    #    'pressure exhaust valve', 'sparge total', 'temperature',
    #    'temperature jacket', 'weight load cell', 'Titer_range', 'Titer_group',
    #    'Titer_category', 'Feed',
        'Glucose (mM)', 'Lactate (mM)', 'Glucose after feeding (mM)', 'feed bool'
        ]

In [38]:
cols_lst =['Osmolarity', 'Viable Cell Concentration (10^6 cells/mL)',
       'Dead Cell Concentration (10^6 cells/mL)',
       'Total Cell Concentration (10^6 cells/mL)',#, 'IgG (mg/L)', 
       'NH4+', 'Na+', 'Titer_category',
        'Glucose (mM)', 'Lactate (mM)', 'Glucose after feeding (mM)']

scale_lst = ["12000L"]
estimating_missing_measuremnts(df, runID_lst,cols_lst, scale_lst=scale_lst)
# for col_name in cols_lst:
#     for i in range(len(runID_lst))
#         idx_lst = df[df['ID']==runID_lst[i]].index.tolist()
#         outlier_removal_other_parameters("Xv", runID_lst[i], df[df['ID']==runID_lst[i]]['Viable Cell Concentration (10^6 cells/mL)'], window = 10, threshold = 2)

,Date (MM/DD/YY H:MM:SS AM/PM),Cell Line,ID,Name,Initial Volume (mL),Sample Volume (mL),Volume Before Sampling (mL),Volume After Sampling (mL),Feed Media Added (mL),Base Added (mL),...,Titer_range,Titer_group,Titer_category,Glucose (mM),Lactate (mM),Feed Status,# Feed,Glucose after feeding (mM),Lactate after feeding (mM),feed bool
0,2024-01-01 00:00:00,12000L,298806,Genentech,12000000,0,12000000,12000000,0,37.355909,...,0.70-0.75,Bottom 20%,2,60.435062,12.997198,0,4,60.435062,12.997198,False
1,2024-01-01 10:00:00,12000L,298806,Genentech,12000000,0,12000000,12000000,0,0.000000,...,0.70-0.75,Bottom 20%,2,60.518113,15.975917,0,4,60.518113,15.975917,False
2,2024-01-01 20:00:00,12000L,298806,Genentech,12000000,0,12000000,12000000,0,0.000000,...,0.70-0.75,Bottom 20%,2,57.799575,19.667813,0,4,57.799575,19.667813,False
3,2024-01-02 06:00:00,12000L,298806,Genentech,12000000,0,12000000,12000000,0,0.000000,...,0.70-0.75,Bottom 20%,2,53.077944,23.575248,0,4,53.077944,23.575248,False
4,2024-01-02 16:00:00,12000L,298806,Genentech,12000000,0,12000000,12000000,0,277.058270,...,0.70-0.75,Bottom 20%,2,48.967738,27.307393,0,4,48.967738,27.307393,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6556,2024-01-10 04:00:00,12000L,757904,Genentech,12000000,0,12000000,12000000,0,26898.900990,...,1.20-1.25,Top 20%,1,24.895863,10.690927,0,1,24.895863,10.690927,False
6557,2024-01-10 14:00:00,12000L,757904,Genentech,12000000,0,12000000,12000000,0,26898.900990,...,1.20-1.25,Top 20%,1,21.288437,13.118959,0,1,21.288437,13.118959,False
6558,2024-01-11 00:00:00,12000L,757904,Genentech,12000000,0,12000000,12000000,0,26898.900990,...,1.20-1.25,Top 20%,1,20.182541,15.546992,0,1,20.182541,15.546992,False
6559,2024-01-11 10:00:00,12000L,757904,Genentech,12000000,0,12000000,12000000,0,26898.900990,...,1.20-1.25,Top 20%,1,17.712143,17.975024,0,1,17.712143,17.975024,False


In [39]:
# calculate cumulative lactate consumption
df['Cumulative Lactate (mmol)'] = df['Lactate (mM)'].copy()
for scale in scale_lst:
    for i in range(len(runID_lst)):
        df['Cumulative Lactate (mmol)'] = cumulative_calculation(
            df['Cumulative Lactate (mmol)'], df[(df['ID']== runID_lst[i])&(df['Cell Line']== scale)]['Lactate (mM)'],
            df[(df['ID']== runID_lst[i])&(df['Cell Line']== scale)]['Volume Before Sampling (mL)'], df[(df['ID']== runID_lst[i])&(df['Cell Line']== scale)]['Volume After Sampling (mL)'],
            prod = True)

# calculate cumulative lactate consumption
df['Cumulative Glucose (mmol)'] = df['Glucose (mM)'].copy()
for scale in scale_lst:
    for i in range(len(runID_lst)):
        df['Cumulative Glucose (mmol)'] = cumulative_calculation(
            df['Cumulative Glucose (mmol)'], df[(df['ID']== runID_lst[i])&(df['Cell Line']== scale)]['Glucose (mM)'],
            df[(df['ID']== runID_lst[i])&(df['Cell Line']== scale)]['Volume Before Sampling (mL)'], df[(df['ID']== runID_lst[i])&(df['Cell Line']== scale)]['Volume After Sampling (mL)'],
            prod = False)
    
# calculate biomass production
df['Cumulative viable biomass (cells)'] = df['Viable Cell Concentration (10^6 cells/mL)'].copy()
for scale in scale_lst:
    for i in range(len(runID_lst)):
        df['Cumulative viable biomass (cells)'] = cumulative_calculation(
            df['Cumulative viable biomass (cells)'], df[(df['ID']== runID_lst[i])&(df['Cell Line']== scale)]['Viable Cell Concentration (10^6 cells/mL)'],
            df[(df['ID']== runID_lst[i])&(df['Cell Line']== scale)]['Volume Before Sampling (mL)'], df[(df['ID']== runID_lst[i])&(df['Cell Line']== scale)]['Volume After Sampling (mL)']
            , prod = True)
    
df['Cumulative dead biomass (cells)'] = df['Dead Cell Concentration (10^6 cells/mL)'].copy()
for scale in scale_lst:
    for i in range(len(runID_lst)):
        df['Cumulative dead biomass (cells)'] = cumulative_calculation(
            df['Cumulative dead biomass (cells)'], df[(df['ID']== runID_lst[i])&(df['Cell Line']== scale)]['Dead Cell Concentration (10^6 cells/mL)'],
            df[(df['ID']== runID_lst[i])&(df['Cell Line']== scale)]['Volume Before Sampling (mL)'], df[(df['ID']== runID_lst[i])&(df['Cell Line']== scale)]['Volume After Sampling (mL)']
            , prod = True)
    
df['Cumulative total biomass (cells)'] = df['Total Cell Concentration (10^6 cells/mL)'].copy()
for scale in scale_lst:
    for i in range(len(runID_lst)):
        df['Cumulative total biomass (cells)'] = cumulative_calculation(
            df['Cumulative total biomass (cells)'], df[(df['ID']== runID_lst[i])&(df['Cell Line']== scale)]['Total Cell Concentration (10^6 cells/mL)'],
            df[(df['ID']== runID_lst[i])&(df['Cell Line']== scale)]['Volume Before Sampling (mL)'], df[(df['ID']== runID_lst[i])&(df['Cell Line']== scale)]['Volume After Sampling (mL)']
            , prod = True)

In [40]:
# Examples code for processing the data for the production scale
# When adding to package, create a wrapper function and take agruements that denote species that need to be smoothed

# outlier removal
for scale in scale_lst:
    for i in range(len(runID_lst)):
        idx_lst = df[df['ID']==runID_lst[i]].index.tolist()
        # # glucose
        df.loc[idx_lst, 'Glucose (mM)'] = outlier_removal_species(
            scale,
            "glucose", runID_lst[i],#runID_lst[i],
            df[df['ID']==runID_lst[i]]['Cumulative Glucose (mmol)'], df[df['ID']==runID_lst[i]]['Glucose (mM)'],
            df[df['ID']==runID_lst[i]]['Volume Before Sampling (mL)'], df[df['ID']==runID_lst[i]]['Volume After Sampling (mL)'],
            s_conc_after_feeding = df[df['ID']==runID_lst[i]]['Glucose after feeding (mM)'],
            feed_idx_lst=feed_idx_lst, window=6, threshold=3, always_consumption = True)#, window=4, window=6
        
        # # Lactate
        df.loc[idx_lst, 'Lactate (mM)'] = outlier_removal_species(
        scale,
        "lactate", runID_lst[i],#runID_lst[i],
        df[df['ID']==runID_lst[i]]['Cumulative Lactate (mmol)'], df[df['ID']==runID_lst[i]]['Lactate (mM)'],
        df[df['ID']==runID_lst[i]]['Volume Before Sampling (mL)'], df[df['ID']==runID_lst[i]]['Volume After Sampling (mL)'],
        s_fvol = np.zeros(len(df)),
        s_fconc = np.zeros(len(df)),
        feed_idx_lst=feed_idx_lst, window=6, threshold=3, always_consumption = False)#, window=4, window=5, window=6
        df.loc[idx_lst, 'Lactate (mM)'] = outlier_removal_other_parameters(scale,"lactate", runID_lst[i], df[df['ID']==runID_lst[i]]['Lactate (mM)'], window = 6, threshold = 3)
        
        # # Xv
        df.loc[idx_lst, 'Viable Cell Concentration (10^6 cells/mL)'] = outlier_removal_other_parameters(scale, "Xv", runID_lst[i], df[df['ID']==runID_lst[i]]['Viable Cell Concentration (10^6 cells/mL)'], window = 6, threshold = 3)

        # # Xd
        df.loc[idx_lst, 'Dead Cell Concentration (10^6 cells/mL)'] = outlier_removal_other_parameters(scale, "Xd", runID_lst[i], df[df['ID']==runID_lst[i]]['Dead Cell Concentration (10^6 cells/mL)'], window = 6, threshold = 3)

        # # Xt
        df.loc[idx_lst, 'Total Cell Concentration (10^6 cells/mL)'] = outlier_removal_other_parameters(scale,"Xt", runID_lst[i], df[df['ID']==runID_lst[i]]['Total Cell Concentration (10^6 cells/mL)'], window = 6, threshold = 3)
        

<Figure size 640x480 with 0 Axes>

In [41]:
# change the glucose after feeding column with the smoothed values
for i in range(len(df)):
    if i not in feed_idx_lst:
        df.loc[i,'Glucose after feeding (mM)'] = df.loc[i,'Glucose (mM)']

In [42]:
# outlier removal of ammonia, Na+, Osmo and others
col_lst = ['Osmolarity','NH4+', 'Na+', 'pCO2', 'pO2','pHoff']
col_lst_name = ['osmo','ammonia', 'sodium_ion', 'pCO2', 'pO2','pHoff']
for scale in scale_lst:
    for idx, col_name in enumerate(col_lst):
        for i in range(len(runID_lst)):
            idx_lst = df[df['ID']==runID_lst[i]].index.tolist()
            df.loc[idx_lst, col_name] = outlier_removal_other_parameters(scale, col_lst_name[idx], runID_lst[i], df[df['ID']==runID_lst[i]][col_name], window = 10, threshold = 2)


In [43]:
df.to_excel("input_files/outliers_removal/genentech_12000L_smoothed_YAL.xlsx")